In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import itertools
from keras.layers import LSTM
from keras import layers

In [3]:
BTC_01 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-01.csv')
BTC_02 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-02.csv')
BTC_03 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-03.csv')
BTC_04 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-04.csv')
BTC_05 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-05.csv')
BTC_06 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-06.csv')
BTC_07 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-07.csv')
BTC_08 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-08.csv')
BTC_09 = pd.read_csv('BTCUSDC/BTCUSDC-1m-2022-09.csv')

columns = ['open_time', 'open'	, 'high','low',	'close','volume',	'close_time','quote_volume',
'count'	,'taker_buy_volume','taker_buy_quote_volume',	'ignore']

for df in [BTC_01, BTC_02, BTC_03, BTC_04, BTC_05, BTC_06, BTC_07, BTC_08, BTC_09]:
  df.columns = columns

BTC = pd.concat([BTC_01, BTC_02, BTC_03, BTC_04, BTC_05, BTC_06, BTC_07, BTC_08, BTC_09])

btc_data = BTC[['close']]
btc_data.head()

,close
0,46308.00
1,46372.24
2,46332.09
3,46333.53
4,46436.05


In [6]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(btc_data)

def create_sequences(data, sequence_length):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:i+sequence_length]
        y = data[i+sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 90

train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - sequence_length:]

x_train, y_train = create_sequences(train_data, sequence_length)
x_test, y_test = create_sequences(test_data, sequence_length)

In [9]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(LSTM(64))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

model.save_weights('./weights/lstm_weights.h5')

Epoch 1/20
7806/7806 [==============================] - 302s 39ms/step - loss: 2.6341e-04 - val_loss: 3.7762e-06
Epoch 2/20
7806/7806 [==============================] - 293s 38ms/step - loss: 9.6599e-06 - val_loss: 1.9088e-06
Epoch 3/20
7806/7806 [==============================] - 293s 38ms/step - loss: 5.2645e-06 - val_loss: 1.2965e-06
Epoch 4/20
7806/7806 [==============================] - 291s 37ms/step - loss: 4.0926e-06 - val_loss: 2.3265e-06
Epoch 5/20
7806/7806 [==============================] - 293s 38ms/step - loss: 3.7897e-06 - val_loss: 6.8223e-07
Epoch 6/20
7806/7806 [==============================] - 295s 38ms/step - loss: 3.4400e-06 - val_loss: 4.8296e-07
Epoch 7/20
7806/7806 [==============================] - 295s 38ms/step - loss: 3.1567e-06 - val_loss: 3.4502e-06
Epoch 8/20
7806/7806 [==============================] - 295s 38ms/step - loss: 2.9287e-06 - val_loss: 1.9972e-06
Epoch 9/20
7806/7806 [==============================] - 302s 39ms/step - loss: 2.8300e-06 - val_

In [10]:
def predict_future(model, input_data, future_steps):
    predictions = []
    current_input = input_data[-sequence_length:]
    
    for _ in range(future_steps):
        current_input = current_input.reshape((1, sequence_length, 1))
        prediction = model.predict(current_input)
        predictions.append(prediction[0, 0])
        current_input = np.append(current_input[0][1:], prediction[0, 0])
        
    return scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

In [11]:
input_data = scaled_data[-sequence_length:]
predicted_prices = predict_future(model, input_data, future_steps=15)

print(predicted_prices)

1/1 [==============================] - 0s 12ms/step
[[19460.873]
 [19480.172]
 [19498.656]
 [19516.729]
 [19534.205]
 [19550.98 ]
 [19567.021]
 [19582.342]
 [19597.004]
 [19611.098]
 [19624.72 ]
 [19637.97 ]
 [19650.945]
 [19663.723]
 [19676.365]]
